In [1]:
from autoplay import AutoPlay
import pandas as pd
import numpy as np
import csv

In [ ]:
means = []
max_ucb = 0
max_performance = 0
max_gain = 0
best_spread = []
for n in range(100):
    ap = AutoPlay()
    ap.init_bot('Basic-Bot', 1000, 50, 4)
    ap.blockPrint()
    ap.run(60, 1000)

    data = pd.read_csv('blackjack_summary_60.csv')
    data = data[data['Deck Size'] < 260]

    win_data = data[data['Win'] > 0]
    los_data = data[data['Loss'] > 0]
    counts = []
    for i in range(int(win_data['True Count'].max())-1):
        w1 = win_data[win_data['True Count'] > i]
        w1 = w1[w1['True Count'] < i+1]
        l1 = los_data[los_data['True Count'] > i]
        l1 = l1[l1['True Count'] < i+1]
        if w1.shape[0] == 0 or l1.shape[0] == 0:
            count = 0
        else:
            count = (w1.shape[0]/l1.shape[0])/(w1.shape[0]+l1.shape[0])*100
        counts.append(count)
    df = pd.DataFrame({'Counts+':counts})
    pdf = df[df['Counts+'] > 0]
    counts = []
    for i in range(-1, int(win_data['True Count'].min()), -1):
        w1 = win_data[win_data['True Count'] > i]
        w1 = w1[w1['True Count'] < i+1]
        l1 = los_data[los_data['True Count'] > i]
        l1 = l1[l1['True Count'] < i+1]
        if w1.shape[0] == 0 or l1.shape[0] == 0:
            count = 0
        else:
            count = (w1.shape[0]/l1.shape[0])/(w1.shape[0]+l1.shape[0])*100
        counts.append(count)
    df = pd.DataFrame({'Counts-':counts})
    mdf = df[df['Counts-'] > 0]
    tdf = pd.DataFrame(pdf['Counts+'])
    tdf.insert(0, 'Counts-', mdf['Counts-'], True)
    bets = []
    for i in range(11):
        bet = tdf.at[i, 'Counts-'] * 100
        bets.append(bet)
    for i in range(11):
        bet = tdf.at[i, 'Counts+'] * 100
        bets.append(bet)
    
    file_to_open = 'blackjack_bet_spread.csv'
    with open(file_to_open, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Bet Spread'])
        writer.writerow(bets)

    ap = AutoPlay()
    ap.load_spread()
    ap.init_bot('Basic-Bot', 1000, 50, 3)
    ap.init_bot('Biased-Bot', 1000, 50, 4)
    ap.blockPrint()
    ap.evaluate_spreads(60, 100)
    
    sdata = pd.DataFrame({'Performance': ap.performances, 'Gain': ap.gains})    
    if sdata.mean()[0] > max_performance:
        max_performance = sdata.mean()[0]
        max_gain = sdata.mean()[1]
        best_spread = bets
    means.append(sdata.mean())
means

In [ ]:
file_to_open = 'blackjack_bet_spread.csv'
with open(file_to_open, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Bet Spread'])
    writer.writerow(best_spread)

ap = AutoPlay()
ap.load_spread() 
ap.init_bot('Basic-Bot', 1000, 50, 3)
ap.init_bot('Biased-Bot', 1000, 50, 4)
ap.blockPrint()
ap.evaluate_spreads(60, 1)
ap.enablePrint()

sdata = pd.DataFrame({'Performance': ap.performances, 'Gain': ap.gains})
sdata.mean()

In [ ]:
best_spread

In [ ]:
max_performance